# <font color="#418FDE" size="6.5" uppercase>**Building Multi Step Chains**</font>

>Last update: 20260118.
    
By the end of this Lecture, you will be able to:
- Describe the difference between simple and multi-step LangChain chains. 
- Implement a multi-step chain that orchestrates several Llama 3 calls for a composite task. 
- Handle intermediate outputs with basic parsing and validation before passing them to subsequent steps. 


## **1. Multi Step Chain Types**

### **1.1. Sequential Chain Basics**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_03/Lecture_A/image_01_01.jpg?v=1768765754" width="250">



>* Sequential chains run ordered, assembly-line style steps
>* Break complex tasks into controllable, inspectable stages

>* Sequential chains mirror real-world staged workflows
>* Each step passes needed info, enabling focused oversight

>* Sequential chains turn vague outcomes into explicit steps
>* They control intermediates, improving transparency and debugging



In [ ]:
#@title Python Code - Sequential Chain Basics

# This script shows a simple sequential text processing pipeline example.
# Each step transforms text and passes results to the next step.
# It illustrates how sequential chains differ from single one-shot calls.

# Example requires only standard Python libraries available in Google Colab.
# No external installations are needed for running this simple demonstration.

# Define a first step that extracts market segments from a short description.
def step_one_extract_segments(problem_description):
    """Return a list of simple market segments from description text."""
    words = problem_description.lower().split()
    segments = []
    if "students" in words:
        segments.append("college students in large cities")
    if "parents" in words:
        segments.append("busy parents with young children")
    if "freelancers" in words:
        segments.append("remote freelancers working from home")
    if not segments:
        segments.append("general adult consumers in urban areas")
    return segments

# Define a second step that summarizes needs for each discovered segment.
def step_two_summarize_needs(segments):
    """Return bullet style need summaries for each provided segment."""
    summaries = []
    for segment in segments:
        summary = f"Segment: {segment} | Needs: quick information, clear pricing, trustworthy reviews."
        summaries.append(summary)
    return summaries

# Define a third step that combines summaries into a short final report.
def step_three_build_report(need_summaries):
    """Return a short report that synthesizes previous need summaries."""
    header = "Mini Market Analysis Report for New Online Service"
    body_lines = []
    for summary in need_summaries:
        body_lines.append(f"- {summary}")
    body = "\n".join(body_lines)
    conclusion = "Overall, focus on convenience, transparency, and reliable customer support across segments."
    report = f"{header}\n\n{body}\n\n{conclusion}"
    return report

# Define a helper that runs all steps sequentially, passing outputs forward.
def run_sequential_chain(problem_description):
    """Run three ordered steps and return intermediate plus final results."""
    segments = step_one_extract_segments(problem_description)
    need_summaries = step_two_summarize_needs(segments)
    report = step_three_build_report(need_summaries)
    return segments, need_summaries, report

# Provide a simple problem description similar to lecture examples.
problem_description = "We are launching a new budgeting app for students, parents, and freelancers."

# Run the sequential chain and capture intermediate outputs for inspection.
segments, need_summaries, final_report = run_sequential_chain(problem_description)

# Print intermediate results to show transparent multi step processing.
print("Step 1 - Extracted segments:")
print(segments)
print("\nStep 2 - Need summaries:")
for summary in need_summaries:
    print(summary)
print("\nStep 3 - Final combined report:")
print(final_report)



### **1.2. Branching and Routing Basics**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_03/Lecture_A/image_01_02.jpg?v=1768765782" width="250">



>* Chains can follow different paths based on context
>* Router chooses the best sub-chain or tool

>* Branching classifies user intent, then selects sub-chain
>* Like specialists, routes tasks to best workflow

>* Chains change path based on intermediate results
>* Enables adaptive workflows for feedback and tutoring



In [ ]:
#@title Python Code - Branching and Routing Basics

# Demonstrate simple branching and routing decisions with plain Python functions.
# Show how different inputs follow different processing paths dynamically.
# Mimic LangChain style routing without external libraries or complex setup.
# pip install langchain llama-index transformers accelerate bitsandbytes.

# Define a simple router that decides processing path based on user intent.
intent_keywords = {"weather": ["weather", "rain", "sunny"], "email": ["email", "invite", "meeting"], "code": ["bug", "error", "function"]}

# Define a function that classifies intent using keyword matching logic.
def classify_intent(user_text):
    for intent, keywords in intent_keywords.items():
        for word in keywords:
            if word in user_text.lower():
                return intent
    return "general"

# Define a handler for weather related questions or conversational requests.
def handle_weather(user_text):
    return "Weather branch chosen: Expect mild temperatures around seventy degrees Fahrenheit."

# Define a handler for email drafting or professional writing tasks.
def handle_email(user_text):
    return "Email branch chosen: Drafting a polite invitation for your upcoming meeting."

# Define a handler for code explanation or debugging style questions.
def handle_code(user_text):
    return "Code branch chosen: Explaining the function step by step in plain language."

# Define a handler for general small talk or uncategorized user requests.
def handle_general(user_text):
    return "General branch chosen: Having a friendly conversation about your question today."

# Define the router that selects which branch function should run next.
def route_request(user_text):
    intent = classify_intent(user_text)
    if intent == "weather":
        return handle_weather(user_text)
    if intent == "email":
        return handle_email(user_text)
    if intent == "code":
        return handle_code(user_text)
    return handle_general(user_text)

# Prepare several example requests that will follow different branches.
examples = ["Will it rain tomorrow in my city?", "Please write an email meeting invite.", "Why does this Python function show an error?", "Tell me something interesting about space exploration."]

# Run the router for each example and print chosen branch results.
for text in examples:
    result = route_request(text)
    print(f"Input: {text} -> {result}")



### **1.3. When to Chain**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_03/Lecture_A/image_01_03.jpg?v=1768765809" width="250">



>* Use chains when tasks split into stages
>* Separate stages simplify prompts, debugging, and improvement

>* Use chains when checking intermediate model outputs
>* Chaining enables validation, enrichment, and safer decisions

>* Use single calls for short, simple tasks
>* Choose chains when modular control outweighs complexity



## **2. Composite LLM Workflows**

### **2.1. Layered Extraction Summaries**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_03/Lecture_A/image_02_01.jpg?v=1768765827" width="250">



>* Break complex understanding into sequential focused passes
>* Each layer extracts, groups, then summarizes information

>* Each layer does a focused, specific task
>* Structured intermediate outputs improve reasoning and debugging

>* Useful for long, messy, real-world documents
>* Multiple layers transform text into decision-ready summaries



In [ ]:
#@title Python Code - Layered Extraction Summaries

# Demonstrate layered extraction summaries using simple Python lists and dictionaries.
# Show three processing layers transforming messy support logs into structured summaries.
# Keep everything beginner friendly and runnable directly inside Google Colab.

# pip install libraries here if needed, but standard Colab Python is sufficient.

# Define a messy multi line customer support transcript string example.
raw_transcript = """Customer: My internet speed is very slow since yesterday night.
Agent: When did the slowdown start exactly, and which city are you in now.
Customer: I am in Dallas, Texas, and it started around 9 PM yesterday.
Agent: I see similar complaints from your area about slow download speeds.
Customer: Also my video calls keep freezing during important business meetings.
Agent: Thanks, I will escalate this performance issue to our network engineering team.
Customer: Last month I had a billing error where I was double charged.
Agent: I refunded that extra charge and noted the billing problem in your account.
Customer: Overall I feel frustrated because issues keep returning every few weeks.
Agent: I understand your frustration and will monitor your connection for several days."""

# First layer extracts simple problem statements from the raw transcript text.
first_layer_problems = [
    {
        "problem": "Slow internet speed since yesterday night.",
        "category": "performance",
        "speaker": "customer",
    },
]

# Extend first layer with additional structured problem dictionaries for variety.
first_layer_problems.append(
    {
        "problem": "Video calls freezing during important business meetings.",
        "category": "performance",
        "speaker": "customer",
    }
)

# Add a billing related problem entry to the first layer list.
first_layer_problems.append(
    {
        "problem": "Billing error with double charge last month.",
        "category": "billing",
        "speaker": "customer",
    }
)

# Second layer groups problems by category into a new dictionary structure.
layer_two_grouped = {}
for item in first_layer_problems:
    category = item["category"]
    if category not in layer_two_grouped:
        layer_two_grouped[category] = []
    layer_two_grouped[category].append(item["problem"])

# Third layer builds concise summaries for each category using simple string joins.
layer_three_summaries = {}
for category, problems in layer_two_grouped.items():
    joined = "; ".join(problems)
    summary = f"Category {category} includes issues like: {joined}"
    layer_three_summaries[category] = summary

# Print a short view of first layer structured extraction results.
print("First layer extracted problems:")
for item in first_layer_problems:
    print(f"- {item['category']} problem: {item['problem']}")

# Print grouped categories from the second processing layer dictionary.
print("\nSecond layer grouped by category:")
for category, problems in layer_two_grouped.items():
    print(f"- {category} has {len(problems)} problems recorded")

# Print final layered summaries that resemble decision ready overviews.
print("\nThird layer concise summaries:")
for category, summary in layer_three_summaries.items():
    print(f"- {summary}")



### **2.2. Turning Summaries Into Actions**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_03/Lecture_A/image_02_02.jpg?v=1768765860" width="250">



>* Summaries become inputs for planning-focused Llama calls
>* Model turns summaries into detailed, executable action plans

>* Define clear action types and output format
>* Second call reasons on summaries to plan work

>* Summaries feed automated tools and downstream systems
>* Second Llama call outputs structured, executable instructions



In [ ]:
#@title Python Code - Turning Summaries Into Actions

# Demonstrate turning summaries into concrete action plans using simple Python structures.
# Simulate two Llama 3 style steps without external APIs or complex setup.
# Show how a summary feeds an action planner that prints a clear checklist.

# pip install langchain-openai llama-index transformers accelerate bitsandbytes.

# Define a function that simulates a previous Llama summary step.
def summarize_issue(long_email_text):
    # Return a compact structured summary dictionary with key details extracted.
    summary = {
        "problem": "User cannot log into account after password reset attempt.",
        "tone": "frustrated but still polite and open to help.",
        "constraints": "Needs resolution within twenty four hours due to travel.",
    }
    return summary

# Define a function that turns the summary into concrete action steps.
def plan_actions_from_summary(summary):
    # Build a list of ordered actions based on summary fields content.
    actions = []
    actions.append("Acknowledge frustration and thank user for clear explanation.")
    actions.append("Verify account ownership and recent password reset activity logs.")
    actions.append("Check for account locks or suspicious login attempts flags.")
    actions.append("Prioritize resolution within twenty four hours due to travel.")
    actions.append("Send reply matching polite tone with clear next steps.")
    return actions

# Example long email text that would normally be summarized by Llama.
long_email = (
    "Hi, I tried resetting my password last night and now I cannot log in. "
    "I am flying tomorrow morning and really need access to my boarding "
    "passes and travel documents. I am pretty frustrated but I know these "
    "things happen. Please help me get back into my account as soon as possible."
)

# First step simulates Llama summary call using the helper function.
summary_result = summarize_issue(long_email)

# Second step simulates Llama planning call using the summary result.
action_plan = plan_actions_from_summary(summary_result)

# Print the summary to show compact understanding before planning actions.
print("Summary used for planning:")
print(f"Problem: {summary_result['problem']}")
print(f"Tone: {summary_result['tone']}")
print(f"Constraints: {summary_result['constraints']}")

# Print the resulting action checklist that could drive downstream tools.
print("\nPlanned support actions checklist:")
for index, step in enumerate(action_plan, start=1):
    print(f"Step {index}: {step}")



### **2.3. Coordinating Multiple Prompts**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_03/Lecture_A/image_02_03.jpg?v=1768765890" width="250">



>* Define each step’s role and needed inputs
>* Sequence outputs like a relay for coherent workflows

>* Limit and structure information passed between steps
>* Use concise, formatted outputs for reliable handoffs

>* Choose sequential, parallel, or iterative task steps
>* Manage dependencies and merge results into workflows



## **3. Intermediate Output Handling**

### **3.1. Parsing structured text**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_03/Lecture_A/image_03_01.jpg?v=1768766192" width="250">



>* Parse model outputs into clear, labeled fields
>* Use markers to separate sections for later steps

>* Plan expected structure and parse around headings
>* Convert messy text into consistent, named fields

>* Plan for messy, inconsistent model outputs
>* Use flexible parsing, defaults, and validation checks



In [ ]:
#@title Python Code - Parsing structured text

# Demonstrate parsing structured text into useful Python fields.
# Show simple extraction using predictable labels and separators.
# Handle minor format variations with forgiving parsing logic.
# pip install some_required_library_if_needed.

# Define a fake model response with labeled structured sections.
model_response = (
    "Issue Description: Laptop overheats after twenty minutes of gaming. "
    "Urgency Level: High. "
    "Recommended Action: Clean vents and reduce graphics settings."
)

# Define a second response with slightly different labels and extra text.
alt_response = (
    "Customer says: fan is very loud. "
    "Problem: Fan noise during normal browsing. "
    "Priority: Medium. "
    "Suggested Fix: Update drivers and check dust buildup."
)

# Create a list of possible label variants for each desired field.
label_variants = {
    "issue": ["Issue Description", "Problem"],
    "urgency": ["Urgency Level", "Priority"],
    "action": ["Recommended Action", "Suggested Fix"],
}

# Define a function that extracts field text following any matching label.
def extract_field(text, labels):
    for label in labels:
        search = label + ":"
        if search in text:
            start = text.index(search) + len(search)
            end = text.find(".", start)
            if end == -1:
                end = len(text)
            return text[start:end].strip()
    return "UNKNOWN"

# Parse both responses into structured dictionaries using the helper function.
parsed_one = {
    "issue": extract_field(model_response, label_variants["issue"]),
    "urgency": extract_field(model_response, label_variants["urgency"]),
    "action": extract_field(model_response, label_variants["action"]),
}

# Parse the alternative response that uses different labels and extra commentary.
parsed_two = {
    "issue": extract_field(alt_response, label_variants["issue"]),
    "urgency": extract_field(alt_response, label_variants["urgency"]),
    "action": extract_field(alt_response, label_variants["action"]),
}

# Print the original responses and the parsed structured representations.
print("Original response one:")
print(model_response)
print("\nParsed fields one:")
print(parsed_one)
print("\nOriginal response two:")
print(alt_response)
print("\nParsed fields two:")
print(parsed_two)



### **3.2. Validation Essentials**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_03/Lecture_A/image_03_02.jpg?v=1768766225" width="250">



>* Validation checks structure and task-appropriate content
>* Prevents early errors from corrupting later steps

>* Layer checks from basic to domain-specific rules
>* Apply structural and business rules to outputs

>* Design chains with validation as a core feature
>* Define rules, auto-correct or reject, log issues



In [ ]:
#@title Python Code - Validation Essentials

# Demonstrate simple validation for intermediate chain outputs before next processing step.
# Show structural checks and domain rules for a travel planning style intermediate result.
# Print whether each intermediate output is safe to pass to the next chain step.

# pip install langchain llama-index sentence-transformers transformers accelerate bitsandbytes.

# Define allowed cities and allowed travel classes for validation.
ALLOWED_CITIES = {"New York", "Chicago", "Los Angeles", "Houston"}
ALLOWED_CLASSES = {"economy", "business", "first"}

# Define a function that validates structure and domain specific rules.
def validate_trip(candidate):
    # Check required keys exist and candidate is a dictionary structure.
    required_keys = {"from_city", "to_city", "date", "travel_class"}
    if not isinstance(candidate, dict):
        return False, "Output is not dictionary structure."

    # Check all required keys are present inside the candidate dictionary.
    if not required_keys.issubset(candidate.keys()):
        return False, "Missing required travel fields detected."

    # Extract fields and normalize travel class for consistent comparison.
    from_city = candidate["from_city"]
    to_city = candidate["to_city"]
    date_text = candidate["date"]
    travel_class = str(candidate["travel_class"]).lower()

    # Check cities are allowed and not identical for sensible travel.
    if from_city not in ALLOWED_CITIES or to_city not in ALLOWED_CITIES:
        return False, "City not allowed within configured list."

    # Ensure departure and arrival cities are different locations.
    if from_city == to_city:
        return False, "Departure and arrival cities identical."

    # Check travel class is one of the allowed categories.
    if travel_class not in ALLOWED_CLASSES:
        return False, "Travel class not within allowed categories."

    # Perform simple date pattern check for basic structural correctness.
    if len(date_text) != 10 or date_text[4] != "-" or date_text[7] != "-":
        return False, "Date format not matching expected pattern."

    # If all checks pass then candidate is considered valid for next step.
    return True, "Trip data valid for next chain step."

# Simulate three model outputs representing intermediate chain results.
model_outputs = [
    {"from_city": "New York", "to_city": "Chicago", "date": "2026-07-04", "travel_class": "Economy"},
    {"from_city": "Houston", "to_city": "Houston", "date": "2026-07-04", "travel_class": "business"},
    {"from_city": "Boston", "to_city": "Chicago", "date": "July fourth", "travel_class": "economy"},
]

# Loop through outputs, validate each, and decide next action.
for index, output in enumerate(model_outputs, start=1):
    is_valid, message = validate_trip(output)
    status = "ACCEPTED" if is_valid else "REJECTED"
    print(f"Output {index} status {status}: {message}")



### **3.3. Robust Error Handling**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_03/Lecture_A/image_03_03.jpg?v=1768766258" width="250">



>* Expect imperfect intermediate outputs and validate them
>* Use checks and controlled fallbacks instead of crashing

>* Separate minor, fixable issues from critical failures
>* Use retries, escalation, or flags for problems

>* Log intermediate outputs, validations, and errors consistently
>* Use trends to refine prompts and safeguards



In [ ]:
#@title Python Code - Robust Error Handling

# Demonstrate robust error handling for multi step text processing chain outputs.
# Show validation, recovery, and escalation when intermediate fields are problematic.
# Keep everything simple, beginner friendly, and clearly printed for inspection.

# pip install langchain llama-index transformers accelerate bitsandbytes.

# Define a fake model step that sometimes returns bad structured data.
import random

def fake_model_step(email_text):
    # Randomly decide which error scenario to simulate.
    scenario = random.choice(["ok", "missing", "bad_urgency", "empty_issue"])

    # Start with a reasonable default structured dictionary output.
    result = {"name": "Alex Smith", "issue": "Laptop overheating", "urgency": "medium"}

    # Simulate missing field scenario by deleting urgency key entirely.
    if scenario == "missing":
        result.pop("urgency", None)

    # Simulate bad urgency scenario using unexpected category string value.
    if scenario == "bad_urgency":
        result["urgency"] = "super_critical_level_five"

    # Simulate empty issue scenario using whitespace only description string.
    if scenario == "empty_issue":
        result["issue"] = "   "

    # Return both result and scenario for easier demonstration printing.
    return result, scenario

# Define validation function that checks fields and returns status plus cleaned data.
def validate_intermediate_output(raw_dict):
    # Define allowed urgency categories for downstream chain steps.
    allowed_urgencies = ["low", "medium", "high"]

    # Initialize cleaned dictionary and list for accumulating error messages.
    cleaned = {}
    errors = []

    # Validate name presence and basic non empty content.
    name = raw_dict.get("name", "").strip()
    if not name:
        errors.append("Missing customer name field value.")
    else:
        cleaned["name"] = name

    # Validate issue presence and non whitespace content.
    issue = raw_dict.get("issue", "").strip()
    if not issue:
        errors.append("Missing or empty issue description field.")
    else:
        cleaned["issue"] = issue

    # Validate urgency presence and membership within allowed categories.
    urgency = raw_dict.get("urgency", "").strip().lower()
    if not urgency:
        errors.append("Missing urgency field value entirely.")
    elif urgency not in allowed_urgencies:
        errors.append("Urgency value outside allowed categories list.")
    else:
        cleaned["urgency"] = urgency

    # Decide severity based on number and type of accumulated errors.
    if not errors:
        status = "ok"
    elif len(errors) == 1 and "urgency" in errors[0].lower():
        status = "recoverable"
    else:
        status = "critical"

    # Return status, cleaned dictionary, and list of error messages.
    return status, cleaned, errors

# Define handler that decides recovery, fallback, or escalation behavior.
def handle_output(status, cleaned, errors):
    # If everything validated correctly, continue chain normally.
    if status == "ok":
        message = "Proceeding normally with validated fields dictionary."
        return message, cleaned

    # If recoverable, apply simple fallback default urgency value.
    if status == "recoverable":
        cleaned["urgency"] = "medium"
        message = "Applied default urgency after minor validation problem."
        return message, cleaned

    # For critical issues, stop chain and escalate with clear explanation.
    escalation_note = "Stopping chain, logging errors, requesting human review escalation."
    return escalation_note, None

# Run demonstration once to keep printed output short and readable.
if __name__ == "__main__":
    # Simulate upstream model step producing possibly flawed intermediate output.
    raw_output, scenario = fake_model_step("Customer support email text placeholder.")

    # Validate intermediate output before passing into next chain step.
    status, cleaned, errors = validate_intermediate_output(raw_output)

    # Decide how to respond based on validation status and error severity.
    action_message, final_payload = handle_output(status, cleaned, errors)

    # Print concise observability information for debugging and monitoring.
    print("Simulated scenario:", scenario)
    print("Raw model output:", raw_output)
    print("Validation status:", status)
    print("Validation errors:", errors)
    print("Action decision:", action_message)
    print("Payload forwarded downstream:", final_payload)



# <font color="#418FDE" size="6.5" uppercase>**Building Multi Step Chains**</font>


In this lecture, you learned to:
- Describe the difference between simple and multi-step LangChain chains. 
- Implement a multi-step chain that orchestrates several Llama 3 calls for a composite task. 
- Handle intermediate outputs with basic parsing and validation before passing them to subsequent steps. 

In the next Lecture (Lecture B), we will go over 'Conversational Memory'